## Setup

In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import random

data_dir = "Downloads/dermavqa"

out_dir = data_dir
outputdirectory = Path(f"./data_output/{out_dir}")

## Load Documents

In [2]:
import pandas as pd

file_path = 'preprocessed_responses_with_scispacy.csv'
df = pd.read_csv(file_path)

response_texts = df['response_text']

In [3]:
df.head()

,id,image_ids,response_text,source,case_folding,no_special_chars,digits_to_words,tokenized,no_stopwords,lemmatized,lemmatized_str,ner_scispacy
0,11mk4th,11mk4th,Most probably it is a case of Milia. A milium ...,ImageCLEF,most probably it is a case of milia. a milium ...,most probably it is a case of milia a milium i...,most probably it is a case of milia a milium i...,"['most', 'probably', 'it', 'is', 'a', 'case', ...","['probably', 'case', 'milia', 'milium', 'small...","['probably', 'case', 'milium', 'milium', 'smal...",probably case milium milium small cyst contain...,"[('case milium', 'ENTITY'), ('milium small cys..."
1,ih99w9,ih99w9,"Most probably it is a case of inflamed pimple,...",ImageCLEF,"most probably it is a case of inflamed pimple,...",most probably it is a case of inflamed pimple ...,most probably it is a case of inflamed pimple ...,"['most', 'probably', 'it', 'is', 'a', 'case', ...","['probably', 'case', 'inflamed', 'pimple', 'ma...","['probably', 'case', 'inflamed', 'pimple', 'ma...",probably case inflamed pimple may due follicul...,"[('pimple', 'ENTITY'), ('folliculitis', 'ENTIT..."
2,11n62qx,11n62qx,"Most probably it is a case of solar lentigo, a...",ImageCLEF,"most probably it is a case of solar lentigo, a...",most probably it is a case of solar lentigo a ...,most probably it is a case of solar lentigo a ...,"['most', 'probably', 'it', 'is', 'a', 'case', ...","['probably', 'case', 'solar', 'lentigo', 'harm...","['probably', 'case', 'solar', 'lentigo', 'harm...",probably case solar lentigo harmless condition...,"[('case', 'ENTITY'), ('solar lentigo', 'ENTITY..."
3,vk578x,vk578x,Most propably it is a case of a cyst in the gr...,ImageCLEF,most propably it is a case of a cyst in the gr...,most propably it is a case of a cyst in the gr...,most propably it is a case of a cyst in the gr...,"['most', 'propably', 'it', 'is', 'a', 'case', ...","['propably', 'case', 'cyst', 'groin', 'area', ...","['propably', 'case', 'cyst', 'groin', 'area', ...",propably case cyst groin area cyst noncancerou...,"[('case cyst groin', 'ENTITY'), ('discomfort p..."
4,zmsedx,zmsedx,This is a case of contact eczema most probably...,ImageCLEF,this is a case of contact eczema most probably...,this is a case of contact eczema most probably...,this is a case of contact eczema most probably...,"['this', 'is', 'a', 'case', 'of', 'contact', '...","['case', 'contact', 'eczema', 'probably', 'due...","['case', 'contact', 'eczema', 'probably', 'due...",case contact eczema probably due contact irrit...,"[('case contact eczema', 'ENTITY'), ('contact ..."


In [17]:
import sys
from yachalk import chalk
import json
import client as client
import pandas as pd
import numpy as np
import uuid
import os

def extractConcepts(prompt: str, metadata={}, model="mistral-openorca:latest"):
    SYS_PROMPT = (
        # "Your task is extract the key concepts (and non-personal entities) mentioned in the given context. "
        # "Extract only the most important and atomistic concepts. Break the concepts down to simpler concepts if needed. "
        # "Categorize the concepts in one of the following categories: [event, concept, place, object, document, organisation, condition, misc].\n"
        # "Format your output as a list of json in this format:\n"
        # "[{'entity': 'The Concept', 'importance': '1-5', 'category': 'Type'}]\n"
        "Your task is to extract key dermatology-related concepts and non-personal entities from the given context. Focus on capturing medically relevant, domain-specific information critical to dermatology. Avoid overly general terms unless they provide essential context."

"Break down complex concepts into simpler, more precise components when necessary. Prioritize terms specific to dermatological conditions, symptoms, treatments, diagnostic methods, medications, procedures, anatomical structures, and other related medical knowledge."

"Categorize each concept under one of the following:\n [condition, symptom, treatment, diagnostic test, medication, procedure, risk factor, anatomical structure, object, event, document, misc]\n."
        
    )
    try:
        response, _ = client.generate(model_name=model, system=SYS_PROMPT, prompt=prompt)
        
        if not response:
            raise ValueError("Empty response from the model.")

        result = json.loads(response)
        result = [dict(item, **metadata) for item in result]
        
    except Exception as e:
        print(f"Error during concept extraction: {e}")
        print(f"Response received: {response if 'response' in locals() else 'No response'}")
        result = None
        
    return result

def graphPrompt(input: str, metadata={}, model="mistral-openorca:latest"):
    if model is None:
        model = "mistral-openorca:latest"
    
    # SYS_PROMPT = (
    #     "You are a network graph maker who extracts terms and their relations from a given context. "
    #     "You are provided with a context chunk (delimited by ```). Your task is to extract the ontology "
    #     "of terms mentioned in the given context, represent key concepts, and find relationships.\n"
    #     "Format your output as a list of json in this format:\n"
    #     "[{'node_1': 'A concept', 'node_2': 'Related concept', 'edge': 'relationship'}]\n"
    # )
    SYS_PROMPT = (
    "You are an expert in dermatology knowledge graphs. Your task is to extract terms and their relationships from a given dermatology-related context. Focus only on medically relevant information "
    "critical to dermatology, including conditions, symptoms, treatments, diagnostic tests, medications, procedures, risk factors, and anatomical structures."

    "Avoid extracting overly generic concepts such as 'disease', 'treatment', or 'test'. Instead, capture precise, domain-specific terms related to dermatology."

    "Identify and represent relationships between these concepts, ensuring they are medically meaningful and relevant to dermatology."
    "Avoid extracting the same concept or relationship more than once. Ensure that each relationship is unique and provides distinct information without repetition or rephrasing. Eliminate relationships that add no new value."
    
    "Do NOT generate overly general relationships, such as stating that a condition 'belongs to dermatology' or 'is a medical condition'. "
    "Avoid including descriptive information or commentary as relationships "

    "Extract only factual, domain-specific, concise relationships that provide valuable insights. Each concept and relationship should be precise, relevant, and free of unnecessary general or explanatory information."

    "Output format: A list of JSON objects where each object represents a relationship in the form nothing else is required:\n"
    "[{'node_1': 'A dermatology-specific concept', 'node_2': 'Related dermatology-specific concept', 'edge': 'relationship'}]\n"
)

    USER_PROMPT = f"context: ```{input}``` \n\n output: "
    
    try:
        response, _ = client.generate(model_name=model, system=SYS_PROMPT, prompt=USER_PROMPT)
        
        if not response:
            raise ValueError("Empty response from the model.")
        
        result = json.loads(response)
        result = [dict(item, **metadata) for item in result]
        
    except Exception as e:
        print(f"Error during graph extraction: {e}")
        print(f"Response received: {response if 'response' in locals() else 'No response'}")
        result = None
    
    return result

In [4]:
import uuid
import pandas as pd
import numpy as np
# from .prompts import extractConcepts
# from .prompts import graphPrompt


def documents2Dataframe(documents) -> pd.DataFrame:
    rows = []
    for chunk in documents:
        row = {
            "text": chunk.page_content,
            **chunk.metadata,
            "chunk_id": uuid.uuid4().hex,
        }
        rows = rows + [row]

    df = pd.DataFrame(rows)
    return df


def df2ConceptsList(dataframe: pd.DataFrame) -> list:
    # dataframe.reset_index(inplace=True)
    results = dataframe.apply(
        lambda row: extractConcepts(
            row.text, {"chunk_id": row.chunk_id, "type": "concept"}
        ),
        axis=1,
    )
    # invalid json results in NaN
    results = results.dropna()
    results = results.reset_index(drop=True)

    ## Flatten the list of lists to one single list of entities.
    concept_list = np.concatenate(results).ravel().tolist()
    return concept_list


def concepts2Df(concepts_list) -> pd.DataFrame:
    ## Remove all NaN entities
    concepts_dataframe = pd.DataFrame(concepts_list).replace(" ", np.nan)
    concepts_dataframe = concepts_dataframe.dropna(subset=["entity"])
    concepts_dataframe["entity"] = concepts_dataframe["entity"].apply(
        lambda x: x.lower()
    )

    return concepts_dataframe


def df2Graph(dataframe: pd.DataFrame, model=None) -> list:
    # dataframe.reset_index(inplace=True)
    
    results = dataframe.apply(
        lambda row: graphPrompt(row['response_text'], {"chunk_id": row['id']}, model), axis=1
    )
    # invalid json results in NaN
    results = results.dropna()
    results = results.reset_index(drop=True)

    ## Flatten the list of lists to one single list of entities.
    concept_list = np.concatenate(results).ravel().tolist()
    return concept_list


def graph2Df(nodes_list) -> pd.DataFrame:
    ## Remove all NaN entities
    graph_dataframe = pd.DataFrame(nodes_list).replace(" ", np.nan)
    graph_dataframe = graph_dataframe.dropna(subset=["node_1", "node_2"])
    graph_dataframe["node_1"] = graph_dataframe["node_1"].apply(lambda x: x.lower())
    graph_dataframe["node_2"] = graph_dataframe["node_2"].apply(lambda x: x.lower())

    return graph_dataframe


## Create a dataframe of all the chunks

In [26]:
# from helpers.df_helpers import documents2Dataframe
# df = documents2Dataframe(pages)
# print(df.shape)
# df.head()

## Extract Concepts

In [27]:
# ## This function uses the helpers/prompt function to extract concepts from text
# from helpers.df_helpers import df2Graph
# from helpers.df_helpers import graph2Df

If regenerate is set to True then the dataframes are regenerated and Both the dataframes are written in the csv format so we dont have to calculate them again. 

        dfne = dataframe of edges

        df = dataframe of chunks


Else the dataframes are read from the output directory

In [19]:
import os
import json
import requests

BASE_URL = os.environ.get('OLLAMA_HOST', 'http://localhost:11434')

# Generate a response for a given prompt with a provided model. This is a streaming endpoint, so will be a series of responses.
# The final response object will include statistics and additional data from the request. Use the callback function to override
# the default handler.
# def generate(model_name, prompt, system=None, template=None, context=None, options=None, callback=None):
#     try:
#         url = f"{BASE_URL}/api/generate"
#         payload = {
#             "model": model_name, 
#             "prompt": prompt, 
#             "system": system, 
#             "template": template, 
#             "context": context, 
#             "options": options
#         }
        
#         # Remove keys with None values
#         payload = {k: v for k, v in payload.items() if v is not None}
        
#         with requests.post(url, json=payload, stream=True) as response:
#             response.raise_for_status()
            
#             # Creating a variable to hold the context history of the final chunk
#             final_context = None
            
#             # Variable to hold concatenated response strings if no callback is provided
#             full_response = ""

#             # Iterating over the response line by line and displaying the details
#             for line in response.iter_lines():
#                 if line:
#                     # Parsing each line (JSON chunk) and extracting the details
#                     chunk = json.loads(line)
                    
#                     # If a callback function is provided, call it with the chunk
#                     if callback:
#                         callback(chunk)
#                     else:
#                         # If this is not the last chunk, add the "response" field value to full_response and print it
#                         if not chunk.get("done"):
#                             response_piece = chunk.get("response", "")
#                             full_response += response_piece
#                             print(response_piece, end="", flush=True)
                    
#                     # Check if it's the last chunk (done is true)
#                     if chunk.get("done"):
#                         final_context = chunk.get("context")
            
#             # Return the full response and the final context
#             return full_response, final_context
#     except requests.exceptions.RequestException as e:
#         print(f"An error occurred: {e}")
#         return None, None

def generate(model_name, prompt, system=None, template=None, context=None, options=None, callback=None):
    try:
        url = f"{BASE_URL}/api/generate"
        payload = {
            "model": model_name, 
            "prompt": prompt, 
            "system": system, 
            "template": template, 
            "context": context, 
            "options": options
        }
        
        # Remove keys with None values
        payload = {k: v for k, v in payload.items() if v is not None}
        
        with requests.post(url, json=payload, stream=True) as response:
            response.raise_for_status()
            
            # Creating a variable to hold the context history of the final chunk
            final_context = None
            
            # Variable to hold concatenated response strings if no callback is provided
            full_response = ""

            # Iterating over the response line by line and displaying the details
            for line in response.iter_lines():
                if line:
                    # Parsing each line (JSON chunk) and extracting the details
                    chunk = json.loads(line)
                    
                    # If a callback function is provided, call it with the chunk
                    if callback:
                        callback(chunk)
                    else:
                        # If this is not the last chunk, add the "response" field value to full_response and print it
                        if not chunk.get("done"):
                            response_piece = chunk.get("response", "")
                            full_response += response_piece
                            print(response_piece, end="", flush=True)
                    
                    # Check if it's the last chunk (done is true)
                    if chunk.get("done"):
                        # Print device info (if available)
                        print("Device info (if available):", chunk.get("device", "Not specified"))
                        final_context = chunk.get("context")
            
            # Return the full response and the final context
            return full_response, final_context
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None, None
    except Exception as e:
        print(f"Unhandled error during generation: {e}")
        print(f"Last response received: {response if 'response' in locals() else 'No response'}")
        return None, None


# Create a model from a Modelfile. Use the callback function to override the default handler.
def create(model_name, model_path, callback=None):
    try:
        url = f"{BASE_URL}/api/create"
        payload = {"name": model_name, "path": model_path}
        
        # Making a POST request with the stream parameter set to True to handle streaming responses
        with requests.post(url, json=payload, stream=True) as response:
            response.raise_for_status()

            # Iterating over the response line by line and displaying the status
            for line in response.iter_lines():
                if line:
                    # Parsing each line (JSON chunk) and extracting the status
                    chunk = json.loads(line)

                    if callback:
                        callback(chunk)
                    else:
                        print(f"Status: {chunk.get('status')}")
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

# Pull a model from a the model registry. Cancelled pulls are resumed from where they left off, and multiple
# calls to will share the same download progress. Use the callback function to override the default handler.
def pull(model_name, insecure=False, callback=None):
    try:
        url = f"{BASE_URL}/api/pull"
        payload = {
            "name": model_name,
            "insecure": insecure
        }

        # Making a POST request with the stream parameter set to True to handle streaming responses
        with requests.post(url, json=payload, stream=True) as response:
            response.raise_for_status()

            # Iterating over the response line by line and displaying the details
            for line in response.iter_lines():
                if line:
                    # Parsing each line (JSON chunk) and extracting the details
                    chunk = json.loads(line)

                    # If a callback function is provided, call it with the chunk
                    if callback:
                        callback(chunk)
                    else:
                        # Print the status message directly to the console
                        print(chunk.get('status', ''), end='', flush=True)
                    
                    # If there's layer data, you might also want to print that (adjust as necessary)
                    if 'digest' in chunk:
                        print(f" - Digest: {chunk['digest']}", end='', flush=True)
                        print(f" - Total: {chunk['total']}", end='', flush=True)
                        print(f" - Completed: {chunk['completed']}", end='\n', flush=True)
                    else:
                        print()
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

# Push a model to the model registry. Use the callback function to override the default handler.
def push(model_name, insecure=False, callback=None):
    try:
        url = f"{BASE_URL}/api/push"
        payload = {
            "name": model_name,
            "insecure": insecure
        }

        # Making a POST request with the stream parameter set to True to handle streaming responses
        with requests.post(url, json=payload, stream=True) as response:
            response.raise_for_status()

            # Iterating over the response line by line and displaying the details
            for line in response.iter_lines():
                if line:
                    # Parsing each line (JSON chunk) and extracting the details
                    chunk = json.loads(line)

                    # If a callback function is provided, call it with the chunk
                    if callback:
                        callback(chunk)
                    else:
                        # Print the status message directly to the console
                        print(chunk.get('status', ''), end='', flush=True)
                    
                    # If there's layer data, you might also want to print that (adjust as necessary)
                    if 'digest' in chunk:
                        print(f" - Digest: {chunk['digest']}", end='', flush=True)
                        print(f" - Total: {chunk['total']}", end='', flush=True)
                        print(f" - Completed: {chunk['completed']}", end='\n', flush=True)
                    else:
                        print()
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

# List models that are available locally.
def list():
    try:
        response = requests.get(f"{BASE_URL}/api/tags")
        response.raise_for_status()
        data = response.json()
        models = data.get('models', [])
        return models

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

# Copy a model. Creates a model with another name from an existing model.
def copy(source, destination):
    try:
        # Create the JSON payload
        payload = {
            "source": source,
            "destination": destination
        }
        
        response = requests.post(f"{BASE_URL}/api/copy", json=payload)
        response.raise_for_status()
        
        # If the request was successful, return a message indicating that the copy was successful
        return "Copy successful"

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

# Delete a model and its data.
def delete(model_name):
    try:
        url = f"{BASE_URL}/api/delete"
        payload = {"name": model_name}
        response = requests.delete(url, json=payload)
        response.raise_for_status()
        return "Delete successful"
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

# Show info about a model.
def show(model_name):
    try:
        url = f"{BASE_URL}/api/show"
        payload = {"name": model_name}
        response = requests.post(url, json=payload)
        response.raise_for_status()
        
        # Parse the JSON response and return it
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

def heartbeat():
    try:
        url = f"{BASE_URL}/"
        response = requests.head(url)
        response.raise_for_status()
        return "Ollama is running"
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return "Ollama is not running"



In [22]:
df.shape

(4265, 12)

In [23]:
df1 = df[:150]
df2 = df[150:300]
df3 = df[300:450]
df4 = df[450:600]
df5 = df[600:750]
df6 = df[750:900]
df7 = df[900:1050]
df8 = df[1050:1200]
df9 = df[1200:1350]
df10 = df[1350:1500]
df11 = df[1500:1650]
df12 = df[1650:1800]
df13 = df[1800:1950]
df14 = df[1950:2100]
df15 = df[2100:2250]
df16 = df[2250:2400]
df17 = df[2400:2550]
df18 = df[2550:2700]
df19 = df[2700:2850]
df20 = df[2850:3000]
df21 = df[3000:3150]
df22 = df[3150:3300]
df23 = df[3300:3450]
df24 = df[3450:3600]
df25 = df[3600:3750]
df26 = df[3750:3900]
df27 = df[3900:4050]
df28 = df[4050:4200]
df29 = df[4200:4265]


In [25]:
df2.head()

,id,image_ids,response_text,source,case_folding,no_special_chars,digits_to_words,tokenized,no_stopwords,lemmatized,lemmatized_str,ner_scispacy
150,126zd0j,126zd0j,The image shows a close-up view of human skin ...,ImageCLEF,the image shows a close-up view of human skin ...,the image shows a closeup view of human skin w...,the image shows a closeup view of human skin w...,"['the', 'image', 'shows', 'a', 'closeup', 'vie...","['image', 'shows', 'closeup', 'view', 'human',...","['image', 'show', 'closeup', 'view', 'human', ...",image show closeup view human skin single well...,"[('image', 'ENTITY'), ('closeup', 'ENTITY'), (..."
151,11k8tv2,11k8tv2,The image shows a close-up view of the side of...,ImageCLEF,the image shows a close-up view of the side of...,the image shows a closeup view of the side of ...,the image shows a closeup view of the side of ...,"['the', 'image', 'shows', 'a', 'closeup', 'vie...","['image', 'shows', 'closeup', 'view', 'side', ...","['image', 'show', 'closeup', 'view', 'side', '...",image show closeup view side person face speci...,"[('image', 'ENTITY'), ('closeup', 'ENTITY'), (..."
152,1270lvd,1270lvd,The image shows a close-up view of the crook o...,ImageCLEF,the image shows a close-up view of the crook o...,the image shows a closeup view of the crook of...,the image shows a closeup view of the crook of...,"['the', 'image', 'shows', 'a', 'closeup', 'vie...","['image', 'shows', 'closeup', 'view', 'crook',...","['image', 'show', 'closeup', 'view', 'crook', ...",image show closeup view crook arm bent elbow s...,"[('image', 'ENTITY'), ('closeup', 'ENTITY'), (..."
153,11rpvbf,11rpvbf,The image displays a close-up view of skin wit...,ImageCLEF,the image displays a close-up view of skin wit...,the image displays a closeup view of skin with...,the image displays a closeup view of skin with...,"['the', 'image', 'displays', 'a', 'closeup', '...","['image', 'displays', 'closeup', 'view', 'skin...","['image', 'display', 'closeup', 'view', 'skin'...",image display closeup view skin light complexi...,"[('image display', 'ENTITY'), ('skin light com..."
154,11w5bmt,11w5bmt,The image displays a close-up view of the back...,ImageCLEF,the image displays a close-up view of the back...,the image displays a closeup view of the back ...,the image displays a closeup view of the back ...,"['the', 'image', 'displays', 'a', 'closeup', '...","['image', 'displays', 'closeup', 'view', 'back...","['image', 'display', 'closeup', 'view', 'back'...",image display closeup view back person lower l...,"[('image display', 'ENTITY'), ('leg', 'ENTITY'..."


In [24]:
import os
import numpy as np
import pandas as pd
from pathlib import Path

dfs = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14, df15, 
       df16, df17, df18, df19, df20, df21, df22, df23, df24, df25, df26, df27, df28, df29]

regenerate = True
outputdirectory = Path("./Downloads/dermavqa/data_output")
outputdirectory.mkdir(parents=True, exist_ok=True)

if regenerate:
    for i, df in enumerate(dfs, start=1):
        print(f"Processing DataFrame {i}...")

        concepts_list = df2Graph(df, model='zephyr:latest')
        dfg = graph2Df(concepts_list)

        dfg.replace("", np.nan, inplace=True)
        dfg.dropna(subset=["node_1", "node_2", 'edge'], inplace=True)
        dfg['count'] = 4

        dfg.to_csv(outputdirectory / f"graph{i}.csv", sep="|", index=False)
        df.to_csv(outputdirectory / f"chunks{i}.csv", sep="|", index=False)

        print(f"Saved graph{i}.csv and chunks{i}.csv")

else:
    dfg = pd.read_csv(r"C:\Users\bhav1\Downloads\dermavqa\graph0.csv", sep="|")

print(dfg.shape)
dfg.head()


Processing DataFrame 1...
[
  {
    "node_1": "Milia",
    "node_2": "keratin",
    "edge": "contains"
  },
  {
    "node_1": "Milia",
    "node_2": "small cyst",
    "edge": "is a"
  },
  {
    "node_1": "Milia",
    "node_2": "lesion",
    "edge": "is a"
  },
  {
    "node_1": "Milia",
    "node_2": "skin condition",
    "edge": "is a"
  },
  {
    "node_1": "Milia",
    "node_2": "dermatology-specific condition",
    "edge": "is a"
  },
  {
    "node_1": "Milia",
    "node_2": "patient concern",
    "edge": "may cause"
  },
  {
    "node_1": "Milia",
    "node_2": "further trauma",
    "edge": "is related to"
  },
  {
    "node_1": "de-roofing",
    "node_2": "sterile needle or blade",
    "edge": "can be used for"
  },
  {
    "node_1": "curettage",
    "node_2": "diathermy",
    "edge": "may be used instead of"
],

In JSON format:
```json
[
  {
    "node_1": "Milia",
    "node_2": "keratin",
    "edge": "contains"
  },
  {
    "node_1": "Milia",
    "node_2": "small cyst",
    "ed

AttributeError: 'list' object has no attribute 'lower'

In [ ]:
with open('convert11.txt', 'w') as convert_file: 
     convert_file.write(json.dumps(concepts_list))

## Calculating contextual proximity

In [10]:
def contextual_proximity(df: pd.DataFrame) -> pd.DataFrame:
    
    dfg_long = pd.melt(
        df, id_vars=["chunk_id"], value_vars=["node_1", "node_2"], value_name="node"
    )
    dfg_long.drop(columns=["variable"], inplace=True)
    dfg_wide = pd.merge(dfg_long, dfg_long, on="chunk_id", suffixes=("_1", "_2"))
    self_loops_drop = dfg_wide[dfg_wide["node_1"] == dfg_wide["node_2"]].index
    dfg2 = dfg_wide.drop(index=self_loops_drop).reset_index(drop=True)

    dfg2 = (
        dfg2.groupby(["node_1", "node_2"])
        .agg({"chunk_id": [",".join, "count"]})
        .reset_index()
    )
    dfg2.columns = ["node_1", "node_2", "chunk_id", "count"]
    dfg2.replace("", np.nan, inplace=True)
    dfg2.dropna(subset=["node_1", "node_2"], inplace=True)
 
    dfg2 = dfg2[dfg2["count"] != 1]
    dfg2["edge"] = "contextual proximity"
    return dfg2


dfg2 = contextual_proximity(dfg1)
dfg2.tail()

,node_1,node_2,chunk_id,count,edge
2029,warts,topical cantharidin,"1251tnr,1251tnr,1251tnr,1251tnr,1251tnr",5,contextual proximity
2030,warts,towels or razors,"1251tnr,1251tnr,1251tnr,1251tnr,1251tnr",5,contextual proximity
2039,water-miscible emollient,dry skin,"11htonk,11htonk,11htonk,11htonk,11htonk,11hton...",12,contextual proximity
2041,water-miscible emollient,eczema due to dry skin (xerosis),"11htonk,11htonk,11htonk,11htonk,11htonk",5,contextual proximity
2053,water-miscible emollient,soap,"11htonk,11htonk,11htonk,11htonk",4,contextual proximity


### Merge both the dataframes

In [11]:
dfg = pd.concat([dfg1, dfg2], axis=0)
dfg = (
    dfg.groupby(["node_1", "node_2"])
    .agg({"chunk_id": ",".join, "edge": ','.join, 'count': 'sum'})
    .reset_index()
)
dfg

,node_1,node_2,chunk_id,edge,count
0,acne,inflamed pimple,"ih99w9,ih99w9,ih99w9",contextual proximity,3
1,aging,deep coarse wrinkle,"11m0l9c,11m0l9c,11m0l9c,11m0l9c,11m0l9c,11m0l9...",contextual proximity,9
2,aging,retinoids,"11m0l9c,11m0l9c,11m0l9c,11m0l9c",contextual proximity,4
3,aloe vera,solar lentigo,"11n62qx,11n62qx",contextual proximity,2
4,aloe vera,topical creams available over the counter,"11n62qx,11n62qx,11n62qx",contextual proximity,3
...,...,...,...,...,...
563,warts,towels or razors,"1251tnr,1251tnr,1251tnr,1251tnr,1251tnr",contextual proximity,5
564,water-miscible emollient,dry skin,"11htonk,11htonk,11htonk,11htonk,11htonk,11hton...",contextual proximity,12
565,water-miscible emollient,eczema due to dry skin (xerosis),"11htonk,11htonk,11htonk,11htonk,11htonk",contextual proximity,5
566,water-miscible emollient,soap,"11htonk,11htonk,11htonk,11htonk",contextual proximity,4


## Calculate the NetworkX Graph

In [12]:
nodes = pd.concat([dfg['node_1'], dfg['node_2']], axis=0).unique()
nodes.shape

(126,)

In [13]:
import networkx as nx
G = nx.Graph()

for node in nodes:
    G.add_node(
        str(node)
    )

for index, row in dfg.iterrows():
    G.add_edge(
        str(row["node_1"]),
        str(row["node_2"]),
        title=row["edge"],
        weight=row['count']/4
    )

### Calculate communities for coloring the nodes

In [14]:
communities_generator = nx.community.girvan_newman(G)
top_level_communities = next(communities_generator)
next_level_communities = next(communities_generator)
communities = sorted(map(sorted, next_level_communities))
print("Number of Communities = ", len(communities))
print(communities)

Number of Communities =  11
[['acne', 'anti-dandruff shampoos', 'dandruff', 'dry or itchy scalp (for people with straight or wavy hair)', 'dry scalp', 'folliculitis', 'head lice', 'inflamed pimple', 'itchy scalp', 'oily and greasy flakes', 'scalp itching', 'seborrheic dermatitis', 'severe itching', 'topical hydrocortisone creams or lotions'], ['aging', 'botox injection', 'chemical peeling', 'chronic sun exposure', 'deep coarse wrinkle', 'facial muscle contraction', 'hydration levels', 'laser resurfacing', 'lines and wrinkles', 'microneedling', 'pigment', 'retinoids', 'skin’s texture', 'smoking', 'soft tissue fillers injection'], ['aloe vera', 'avoid sun damage', 'clarify diagnosis', 'creams containing licorice extract', 'creams containing vitamin c', 'dermatoscopy', 'glycolic acid', 'harmless condition', 'high spf sunscreen', 'hydroxy acid', 'kojic acid', 'licorice extract', 'prolonged sun exposure', 'protection against sun damage', 'solar lentigo', 'topical creams available over the c

### Create a dataframe for community colors

In [15]:
import seaborn as sns
palette = "hls"

## Now add these colors to communities and make another dataframe
def colors2Community(communities) -> pd.DataFrame:
    ## Define a color palette
    p = sns.color_palette(palette, len(communities)).as_hex()
    random.shuffle(p)
    rows = []
    group = 0
    for community in communities:
        color = p.pop()
        group += 1
        for node in community:
            rows += [{"node": node, "color": color, "group": group}]
    df_colors = pd.DataFrame(rows)
    return df_colors


colors = colors2Community(communities)
colors

,node,color,group
0,acne,#dba757,1
1,anti-dandruff shampoos,#dba757,1
2,dandruff,#dba757,1
3,dry or itchy scalp (for people with straight o...,#dba757,1
4,dry scalp,#dba757,1
...,...,...,...
121,skin care moisturizers,#c7db57,9
122,irritant or allergen,#7fdb57,10
123,symptoms of eczema,#7fdb57,10
124,itchiness and swelling,#57db77,11


### Add colors to the graph

In [16]:
for index, row in colors.iterrows():
    G.nodes[row['node']]['group'] = row['group']
    G.nodes[row['node']]['color'] = row['color']
    G.nodes[row['node']]['size'] = G.degree[row['node']]

In [17]:
pip install pyvis

Note: you may need to restart the kernel to use updated packages.


In [20]:
from pyvis.network import Network

graph_output_directory = "./index.html"

net = Network(
    notebook=False,
    # bgcolor="#1a1a1a",
    cdn_resources="remote",
    height="900px",
    width="100%",
    select_menu=True,
    # font_color="#cccccc",
    filter_menu=False,
)

net.from_nx(G)
# net.repulsion(node_distance=150, spring_length=400)
net.force_atlas_2based(central_gravity=0.015, gravity=-31)
# net.barnes_hut(gravity=-18100, central_gravity=5.05, spring_length=380)
net.show_buttons(filter_=["physics"])

net.show(graph_output_directory)